In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Set up the canvas size and background color
canvas_size = (28, 28)
background_color = 255  # white background
draw_color = 0  # black drawing color

# Create a blank white image (28x28 pixels)
image = Image.new('L', canvas_size, background_color)
draw = ImageDraw.Draw(image)

# Function to draw on the canvas
def draw_on_canvas(event):
    x, y = event.x, event.y
    # Draw a black dot where the user clicks (can later replace with more sophisticated drawing)
    draw.point((x, y), fill=draw_color)
    plt.imshow(image, cmap='gray')
    plt.draw()

# Set up the plot
fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
ax.set_title("Draw a letter")
plt.show()


In [ ]:
# Convert image to grayscale (already grayscale in this case)
image = image.convert('L')

# Resize image to 28x28 (if not already)
image = image.resize((28, 28))

# Convert to numpy array and normalize
image_array = np.array(image) / 255.0  # Normalize to [0, 1]

# Flatten the image for ANN (if you're using ANN)
image_flattened = image_array.flatten()

# Display the processed image
plt.imshow(image_array, cmap='gray')
plt.show()


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

# Example: Collect data for letters A-Z (assuming you collected the data)
# You will need to store the images and labels in arrays
X_train = []  # List to store the input images
y_train = []  # List to store the labels

# Assume you've manually collected one image for each letter (A-Z)
# Append the preprocessed image and its corresponding label
X_train.append(image_flattened)  # Append your preprocessed image
y_train.append('A')  # Corresponding label (e.g., "A")

# Convert the labels to one-hot encoding
# First, map letters to integers
letter_to_int = {chr(i): i - ord('A') for i in range(ord('A'), ord('Z') + 1)}
y_train_int = [letter_to_int[label] for label in y_train]
y_train_one_hot = to_categorical(y_train_int, num_classes=26)

# Convert X_train to a numpy array
X_train = np.array(X_train)

# Build the ANN model
model = Sequential()
model.add(Dense(128, input_dim=28*28, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(26, activation='softmax'))  # 26 output neurons for 26 letters

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your collected data
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=1)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Reshape the input to (28, 28, 1) for CNN
X_train_cnn = X_train.reshape(-1, 28, 28, 1)

# Build the CNN model
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(26, activation='softmax'))  # 26 output neurons for 26 letters

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(X_train_cnn, y_train_one_hot, epochs=10, batch_size=1)
